<a href="https://colab.research.google.com/github/YahyaAlaaMassoud/learn-search-relevance/blob/main/quora_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import warnings
warnings.filterwarnings('ignore')

Creating requirments.txt file and writing the needed dependencies, then installing them.

In [1]:
%%writefile requirments.txt
datasets

Writing requirments.txt


In [ ]:
!echo "Requirments Are:" && cat requirments.txt && echo "-----"

!pip install -r requirments.txt

In [3]:
import numpy as np

from pprint import pprint

Use hf/datasets library to import the quora datasets

In [4]:
from datasets import load_dataset_builder

load_dataset_builder can help quickly get some general info about the dataset (e.g. features, description, size)

In [5]:
ds_builder = load_dataset_builder('quora') # retruns a parquet file
ds_builder

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
pprint(ds_builder.info)

DatasetInfo(description='The Quora dataset is composed of question pairs, and '
                        'the task is to determine if the questions are '
                        'paraphrases of each other (have the same meaning).',
            citation='',
            homepage='https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs',
            license='',
            features={'is_duplicate': Value(dtype='bool', id=None),
                      'questions': Sequence(feature={'id': Value(dtype='int32',
                                                                 id=None),
                                                     'text': Value(dtype='string',
                                                                   id=None)},
                                            length=-1,
                                            id=None)},
            post_processed=None,
            supervised_keys=None,
            task_templates=None,
            builder_name=

In [7]:
from datasets import get_dataset_config_names

'config_names', get_dataset_config_names('quora')

('config_names', ['default'])

In [8]:
from datasets import get_dataset_split_names

'split_names', get_dataset_split_names('quora')

('split_names', ['train'])

In [9]:
split_range = '[0:5000]'
split = 'train{}'.format(split_range)
'split', split

('split', 'train[0:5000]')

In [10]:
from datasets import load_dataset

quora_ds = load_dataset(path='quora', split=split)
quora_ds

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 5000
})

In [11]:
pprint(quora_ds[0]); pprint(quora_ds[-1]);

{'is_duplicate': False,
 'questions': {'id': [1, 2],
               'text': ['What is the step by step guide to invest in share '
                        'market in india?',
                        'What is the step by step guide to invest in share '
                        'market?']}}
{'is_duplicate': False,
 'questions': {'id': [9858, 9859],
               'text': ['How much does U.S. $500,000 weigh in cash?',
                        'Can you be rich with $500,000?']}}


In [12]:
pprint(quora_ds['questions'][:5])

[{'id': [1, 2],
  'text': ['What is the step by step guide to invest in share market in india?',
           'What is the step by step guide to invest in share market?']},
 {'id': [3, 4],
  'text': ['What is the story of Kohinoor (Koh-i-Noor) Diamond?',
           'What would happen if the Indian government stole the Kohinoor '
           '(Koh-i-Noor) diamond back?']},
 {'id': [5, 6],
  'text': ['How can I increase the speed of my internet connection while using '
           'a VPN?',
           'How can Internet speed be increased by hacking through DNS?']},
 {'id': [7, 8],
  'text': ['Why am I mentally very lonely? How can I solve it?',
           'Find the remainder when [math]23^{24}[/math] is divided by '
           '24,23?']},
 {'id': [9, 10],
  'text': ['Which one dissolve in water quikly sugar, salt, methane and carbon '
           'di oxide?',
           'Which fish would survive in salt water?']}]


In [13]:
pprint(quora_ds['questions'][0]['text'])

['What is the step by step guide to invest in share market in india?',
 'What is the step by step guide to invest in share market?']


In [14]:
pprint(quora_ds[:3])

{'is_duplicate': [False, False, False],
 'questions': [{'id': [1, 2],
                'text': ['What is the step by step guide to invest in share '
                         'market in india?',
                         'What is the step by step guide to invest in share '
                         'market?']},
               {'id': [3, 4],
                'text': ['What is the story of Kohinoor (Koh-i-Noor) Diamond?',
                         'What would happen if the Indian government stole the '
                         'Kohinoor (Koh-i-Noor) diamond back?']},
               {'id': [5, 6],
                'text': ['How can I increase the speed of my internet '
                         'connection while using a VPN?',
                         'How can Internet speed be increased by hacking '
                         'through DNS?']}]}


Compare two types of indexing

In [15]:
import time

start_time = time.time()
pprint(quora_ds[0]['questions']['text'])
print('elapsed time: {}'.format(time.time() - start_time))

start_time = time.time()
pprint(quora_ds['questions'][0]['text'])
print('elapsed time: {}'.format(time.time() - start_time))

['What is the step by step guide to invest in share market in india?',
 'What is the step by step guide to invest in share market?']
elapsed time: 0.0013337135314941406
['What is the step by step guide to invest in share market in india?',
 'What is the step by step guide to invest in share market?']
elapsed time: 0.2001805305480957


A function that gathers all unique questions in a list

In [16]:
def get_questions(ds):
  questions = ds['questions']
  questions_text = set()
  for q in questions:
    for q_text in q['text']:
      questions_text.add(q_text)
  unique_questions = list(questions_text)
  return [{'question': q} for q in unique_questions]

quora_questions = get_questions(ds=quora_ds)
pprint(quora_questions[:5])
'number of questions', len(quora_questions)

[{'question': 'How do I build a self confidence?'},
 {'question': 'Is there a BSc Economics course being offered in India by a top '
              'distance university?'},
 {'question': 'What causes nightmares that seem real?'},
 {'question': 'I had send my resume below.?'},
 {'question': 'Do you think Egyptian government still hides information on '
              'their Pyramids? If yes, why and what would be the outcome if '
              "they didn't?"}]


('number of questions', 9859)

In [17]:
%%writefile -a requirments.txt
sentence-transformers

Appending to requirments.txt


In [ ]:
!echo "Requirments Are:" && cat requirments.txt && echo "-----"

!pip install -r requirments.txt

In [19]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = 'all-MiniLM-L12-v2'

encoder = SentenceTransformer(model_name_or_path=EMBEDDING_MODEL_NAME)
encoder

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [20]:
pprint({
    'max_seq_length': encoder.get_max_seq_length(),
    'sentence_embedding_dimension': encoder.get_sentence_embedding_dimension(),
    'tokenizer': encoder.tokenizer
})

{'max_seq_length': 128,
 'sentence_embedding_dimension': 384,
 'tokenizer': BertTokenizerFast(name_or_path='sentence-transformers/all-MiniLM-L12-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}}


In [21]:
pprint(quora_questions[0]['question'])

'How do I build a self confidence?'


In [22]:
def compute_embedding(sentences, emb_model):
  return emb_model.encode(sentences=sentences)

In [23]:
embedding = compute_embedding(sentences=quora_questions[0]['question'], emb_model=encoder)
assert embedding.shape[0] == encoder.get_sentence_embedding_dimension()
embedding.shape, np.min(embedding), np.max(embedding), np.mean(embedding)

((384,), -0.12946042, 0.17405869, 0.0011720624)

In [24]:
%%writefile -a requirments.txt
qdrant-client

Appending to requirments.txt


In [ ]:
!echo "Requirments Are:" && cat requirments.txt && echo "-----"

!pip install -r requirments.txt

In [26]:
from qdrant_client import QdrantClient, models

qdrant = QdrantClient(':memory:') # create in-mem instance of vector db

In [27]:
qdrant.recreate_collection(
    collection_name='questions',
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.EUCLID
    )
)

True

In [28]:
qdrant.upload_records(
    collection_name='questions',
    records=[
        models.Record(
            id=idx,
            payload=entry,
            vector=compute_embedding(entry['question'], encoder).tolist()
        ) for idx, entry in enumerate(quora_questions)
    ]
)

<ipython-input-28-ee0b017c180f>:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [29]:
def query(question, top_k=5):
  results = qdrant.search(
    collection_name='questions',
    query_vector=compute_embedding(question, encoder),
    limit=top_k
  ) # search for top 5 questions in the quora questions DB similar to the new question
  return results

In [31]:
results = query('obama', top_k=5)

for result in results:
  print('score', result.score, result.payload)

score 0.7964085936546326 {'question': 'Why is Barack Obama so important to the world?'}
score 0.798779308795929 {'question': 'Why is Barack Obama so important for the world?'}
score 0.8063689470291138 {'question': 'What is Barack Obama doing now?'}
score 0.8552512526512146 {'question': 'What do you think about Barack Obama?'}
score 0.8633735775947571 {'question': 'What great achievements will Barack Obama be remembered for after his tenure as president?'}
